# test implementations 

In [8]:
import json
import numpy as np
import pandas as pd

import sklearn as skl
from sklearn import preprocessing
from sklearn.linear_model import Ridge
from sklearn.neighbors import KNeighborsRegressor

#Our implementations
import kNN as OurKNN
import RidgeRegression as OurRidge

cross_validator10 = skl.model_selection.StratifiedKFold(n_splits=10)


#### Dataset 1: Wikipedia maths
Json structure:
{"edges":[[x,y]... array of ordered tuples of links],
"weights":[(ordered) array of weights for the edges, I guess they represent the number of times one topic is linked to another],
"node_ids":{"topic name": id, set of ids for all topics},
"time_periods":731,
"0": {"index":0, "year":2019, "month": 3, "day":16, "y":[124, 1240, 123...]},
"1":{"index":1, ...}
...
}

In [2]:
# Load data, process, potentially plot some distributions

with open('data/wikivital_mathematics.json') as data_file:    
    math_data = json.load(data_file)
    
keys = math_data.keys()

topics_id = math_data["node_ids"]
topics = math_data["node_ids"].keys()

#### Dataset 2: Energy efficiency

In [3]:
# Load data, process, potentially plot some distributions

df_raw = pd.read_csv('data/energy_efficiency_data.csv')
num_entries = df_raw.shape[0]
num_attributes = df_raw.shape[1]

df_shuffeled = df_raw.sample(frac=1,random_state=193520285)

#Preprocess target: Pick one of two possible targets, mean = 0 
Y = df_shuffeled.values[:,-2:-1]
Y = np.reshape(Y, len(Y))
Y = Y - np.mean(Y)

#Preprocess features. Scale all to [0,1]
X = df_shuffeled.values[:,:-2]
scaler = preprocessing.MinMaxScaler()
scaler.fit(X)
df_shuffeled = scaler.transform(X)


In [4]:
# Ridge regression, own and available implementation + plot

learning_rate = 0.000000002
lambda_parameter = 100.00 
max_iterations = 400

# Perform cross validation
for train_index, test_index in cross_validator10.split(X,Y):
    
    # Fit models
    OurRidge.fit(X,Y, lambda_parameter, learning_rate, max_iter = max_iterations, quiet=True)
    
    availableRidge = Ridge(lambda_parameter, fit_intercept=False, max_iter=max_iterations)
    availableRidge.fit(X[train_index],Y[train_index])
    
    #Test models
    

print(availableRidge)

Initial cost:155360635.13
Terminating fitting since relative improvement dropped below threshold.

Final weights: [0.33636366425152375, -0.06186695338041282, 0.13161879382883424, -0.030305257633830296, 0.044554325619518234, 0.44270077048631146, 0.6904230192179557, 0.9464973748774235]
Total size of weights: 1.306032249339568
RSS split of cost: 49.76%
Mean residual: 3.552986673393098, compare to difference between max and min y: 37.09
Ridge(alpha=10000.0, fit_intercept=False, max_iter=400)


In [5]:
# kNN 

OurKNN.fit(X,Y)


#### Dataset 3: